In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm, colors
from pprint import pprint
from scipy.ndimage import gaussian_filter1d
from IPython.display import display

# brainbox / iblatlas / ONE 관련
from brainbox.io.one import SessionLoader, SpikeSortingLoader
from brainbox.singlecell import bin_spikes
from brainbox.ephys_plots import plot_brain_regions
from iblatlas.atlas import AllenAtlas
from one.api import ONE

import os
import sys

BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))

def add_module_paths(base, *rel_paths):
    for rel_path in rel_paths:
        sys.path.append(os.path.join(base, *rel_path))

add_module_paths(BASE_DIR,
    ['func'],               # func 바로 아래 함수들
    ['func', 'compute'],
    ['func', 'info'],
    ['func', 'plot']
)

from compute_raster import compute_raster
from sub_func import save_file

SAVE_PATH = r"C:\Users\miasc\SCH\shinlab\ChaeHyeon_Seong\dataset_v2"


Current working directory: c:\Users\miasc\SCH\shinlab\IBL\VISp_PSTH\main
Python path: ['c:\\Users\\miasc\\anaconda3\\envs\\iblenv\\python39.zip', 'c:\\Users\\miasc\\anaconda3\\envs\\iblenv\\DLLs', 'c:\\Users\\miasc\\anaconda3\\envs\\iblenv\\lib', 'c:\\Users\\miasc\\anaconda3\\envs\\iblenv', '', 'c:\\Users\\miasc\\anaconda3\\envs\\iblenv\\lib\\site-packages', 'c:\\Users\\miasc\\anaconda3\\envs\\iblenv\\lib\\site-packages\\win32', 'c:\\Users\\miasc\\anaconda3\\envs\\iblenv\\lib\\site-packages\\win32\\lib', 'c:\\Users\\miasc\\anaconda3\\envs\\iblenv\\lib\\site-packages\\Pythonwin', '././fucn/', '././fucn/', '././fucn/']


In [ ]:
one = ONE()

brain_acronym = 'VISp'
sessions = list(one.search(atlas_acronym=brain_acronym, query_type='remote'))

for i in [44, 45, 68, 68, 68]: sessions.pop(i)
print(f"Found {len(sessions)} sessions for region: {brain_acronym}")

# ---------------------------------------------------------------
# 세션 반복
# ---------------------------------------------------------------
i = 1
for eid in sessions:

    if i == 10: break # 원하는 세션 수 만큼 반복 후 종료 : 10개

    # Session ID
    print(f"\n=== [Session:{i} {eid}] ==="); i += 1

    # Dataset 내에 Session 폴더 생성
    save_session_path = os.path.join(SAVE_PATH, eid); os.makedirs(save_session_path, exist_ok=True)

    # -----------------------------------------------------------------
    # Save (1) : Dataset 내에 ONE 폴더 내의 Session 위치 저장
    # -----------------------------------------------------------------
    
    with open(os.path.join(save_session_path, "session_path.txt"), "w", encoding="utf-8") as f:
        f.write(str(one.eid2path(eid)))

    # -----------------------------------------------------------------
    # Save (2) : Trials 정보 로드 & 저장
    # -----------------------------------------------------------------
    sl = SessionLoader(eid=eid, one=one)
    sl.load_trials()
    trials_df = sl.trials

    # --- basic task와 full task 구분 ---
    if 'probabilityLeft' in trials_df.columns:
        unique_prob = np.unique(trials_df['probabilityLeft'].dropna()) 
        if len(unique_prob) == 1 and np.isclose(unique_prob[0], 0.5): 
            task_type = 'basic'
        else:
            task_type = 'full'
    else:
        task_type = 'unknown'
    print(f"Session {eid} is identified as a {task_type} task.")
    with open(os.path.join(save_session_path, "task_type.txt"), "w", encoding="utf-8") as f:
        f.write(task_type)

    trials_df_selected = trials_df[['stimOn_times', 'contrastLeft', 'contrastRight',
                                    'choice', 'feedbackType', 'response_times']].copy()
    trials_df_selected['task_type'] = task_type

    save_file(trials_df_selected,
              save_path=save_session_path,
              save_title="trials_info")

    # -----------------------------------------------------------------
    # 2) Spike/Cluster 정보: Probe 별로 반복
    # -----------------------------------------------------------------
    pids, labels = one.eid2pid(eid)
    if len(pids) == 0:
        print(f" - No probe data found in session {eid}, skip.")
        continue

    events = sl.trials['stimOn_times'].values

    for pid, label in zip(pids, labels):
        print(f"   -> Probe: {pid} ({label})")
        probe_folder = os.path.join(save_session_path, label)
        os.makedirs(probe_folder, exist_ok=True)

        clusters_from_obj = one.load_object(eid, 'clusters', collection=f'alf/{label}/pykilosort')
        peak2trough = clusters_from_obj.peakToTrough

        ssl = SpikeSortingLoader(one=one, pid=pid, atlas=AllenAtlas())
        spikes, clusters, channels = ssl.load_spike_sorting()
        clusters = ssl.merge_clusters(spikes, clusters, channels)

        # -----------------------------
        # 2-1) 뉴런 정보와 spike waveform 분류 및 전체 recording FR 계산
        # -----------------------------
        clusters_df = clusters.to_df()
        clusters_df['peakToTrough'] = peak2trough
        threshold = 0.5  # 임계값 (ms)
        clusters_df['spikeType'] = clusters_df['peakToTrough'].abs().apply(
            lambda x: 'fast-spiking' if x < threshold else 'regular-spiking'
        )

        # 전체 recording에서 각 뉴런의 평균 firing rate 계산
        # spikes.times는 session 전체의 spike 시간 (초 단위)
        # 각 unit에 대해 spike 개수를 세고, 전체 recording 기간(마지막 - 첫 spike 시간)으로 나눔
        recording_duration = spikes.times[-1] - spikes.times[0]
        unique_units = np.unique(spikes.clusters)
        mean_fr_dict = {unit: np.sum(spikes.clusters == unit) / recording_duration 
                        for unit in unique_units}
        # clusters_df의 인덱스 혹은 'id' 컬럼을 사용하여 매핑 (여기서는 index를 사용)
        clusters_df['meanFR_session'] = clusters_df.index.map(lambda x: mean_fr_dict.get(x, np.nan))

        save_file(clusters_df, save_path=probe_folder, save_title="neuron_info")
        # 전체 session FR를 별도로 저장 (Method 1용)
        np.save(os.path.join(probe_folder, "mean_fr_session.npy"), 
                clusters_df['meanFR_session'].values)

        # -------------------------------------------------------------
        # 2-2) PSTH 계산 (trial window 기반은 그대로 두되, Method 2, 3 계산은 그대로 수행)
        # -------------------------------------------------------------
        pre_time = 2.0
        post_time = 4.0
        bin_size = 0.001  # 1ms
        # psth의 시간 배열 (-2 ~ +4초)
        # (참고: 이 PSTH는 trial-locked window로 계산됨)
        # times = np.arange(-pre_time, post_time, bin_size)  <- 이미 compute_raster 내부에서 사용됨

        spike_raster, time_bins = compute_raster(spikes, 
                                                  np.where((clusters_df.index.values)[clusters_df.index.isin(unique_units)])[0],
                                                  events,
                                                  pre_time=pre_time,
                                                  post_time=post_time,
                                                  bin_size=bin_size)
        save_file(spike_raster, save_path=probe_folder, save_title="psth")
        save_file(time_bins, save_path=probe_folder, save_title="time_bins")

print("\n\nAll done!")
